In [5]:
%load_ext autoreload
%autoreload 2

In [34]:
import os
os.chdir("C:/users/nathan/documents/dropbox/python/scripts/tools/patcher/test/unittests/data_files/sysvars_case_files")
import sys
sys.path.append("C:/users/nathan/documents/dropbox/python/scripts/tools/patcher/src/merge")

In [2]:
class File():
    def __init__(self, fp, content):
        self.fp = fp
        self.content = content
        
    @classmethod
    def fromother(cls, other, fp):
        with open(other, 'r') as f:
            c = f.read()
        return cls(fp, c)

In [3]:
class TestInfo():
    def __init__(self, patch, cff, off, nff, exp, result, name=None):
        self.patch = patch
        self.cff = cff
        self.nff = nff
        self.off = off
        self.expected = exp
        self._default_output = result
        self.kw = {}
        self._outf_from_file = self.find_output()
        self.result = self._outf_from_file or result
        self.name = name or os.path.basename(patch)

    def get_cmd(self):
        """
        Return the command line arguments for the test case
        to be run from subprocess.Popen() (or equivalent).
        """
        kw = self.kw.copy()

        # else, output is handled in patch file already
        if not self._outf_from_file:
            kw['--outf'] = self._default_output

        # it should be fine just to wrap stuff in quotes ...
        return make_command(self.patch, self.cff, self.off, self.nff, **kw)

    def find_output(self):
        """ Find the name and location of the output of
        merge.exe based on the supplied TestInfo.

        Note - path calculation MUST match the logic used
        by merge/parse_config.py and merge/merge.py !!!
        """
        c = configparser.ConfigParser(allow_no_value=True)
        c.optionxform = str
        
        with open(self.patch, 'r') as f:
            c.read_file(f)

        outf = c['General']['Output'].format(filename=os.path.basename(self.cff))
        if "--outf" in self.kw:
            outf = self.kw['--outf']

        if outf:
            return os.path.abspath(outf)
        else:
            return ""

    def __repr__(self):
        return "<%s: %s>"%(self.__class__.__name__, self.name)

    def cleanup(self):
        self.cleanup_output()

    def cleanup_output(self):
        op = self.find_output()
        if os.path.exists(op):
            os.remove(op)

    def run_merge(self):
        cmd(self.get_cmd())

In [35]:
from collections import OrderedDict as OD
from scripts.tools.patcher.src.merge import lv_parse
from scripts.tools.patcher.src.merge.mergers import merge_sysvars
from scripts.tools.patcher.src.merge.parse_config import Options
import os

def lv2d(lv):
    d = OD()
    for g in lv.val:
        for v in g.val:
            d[g.name+"."+v.name] = v
    return d


def d2lv(d):
    c = lv_parse.LVCluster("System Variables Mag")
    last = lv_parse.LVCluster("dummy")
    for key, value in d.items():
        gn,vn = key.split(".")
        if last.name != gn:
            last = c.get(gn)
            if not last:
                last = lv_parse.LVCluster(gn)
                c.val.append(last)
        last.val.append(value)
    return c

def s2lv(fp):
    return merge_sysvars._parse_sysvars(fp)

In [ ]:
def randomize_vals(d, minc=1, maxc=10):
    keys = list(d.keys())
    ind = list(range(len(keys)))
    random.shuffle(ind)
    n = random.randint(minc, maxc)
    for _ in range(n):
        i = ind.pop()
        k = keys[i]
        v = d[k]
        
sv = s2lv("sysvars_revb_ref.sys")
def sv_case_rnd_vals_simple():
    old  = s2lv("sysvars_revb_ref.sys")
    user = s2lv("sysvars_revb_ref.sys")
    o    = Options()
    
    
case sv_case_simple1
    

In [33]:
def svmake_case(name, info):
    make_case("sysvars", name, info)
    
def make_case(typ, name, info):
    dn = os.path.join(typ+"_case_files", "case_"+name)
    os.makedirs(dn)
    write_file(info.off, dn, typ)
    write_file(info.cff, dn, typ)
    write_file(info.nff, dn, typ)
    write_file(info.expected, dn, typ)